# Анализ новостей

Это домашнее задание будет довольно большим. В нём мы попробуем кучу разных штук. Работать мы будем с датасетом новостей с РИА-новости за 2018 год. Он состоит из двух частей: 

* Новости с сайта https://ria.ru, заботливо собранные мной для вас.
* Новости из оффициальной группы РИА-новости вконтакте, а также комментарии к ним. Они тоже собирались для вас мной с заботой. 

Задание состоит из $4$ частей. Первые две являются обязательными. Последние две опциональными (зависит от вашего терпения). 

1. Нужно обучить на корпусе новостных заголовков рекурентную нейронную сетку и сгенерировать парочку новых, никогда ранее не существовавших новостей. 
2. Каждой новости в соотвествие поставлены ключевые слова. Будем считать, что эти ключевые слова - тематики новости. Нужно научиться предсказывать ключевые слова по тексту новости. Для этого придётся собрать свою собственную архитектуру со своей собственной предобработкой. 
3. __Необязательно:__ Обучите на твиттере сентимент-классификатор. Примените его к комментариям, оставленным под соответсвующими новостями. Выявите топ позитивных новостей и топ негативных. Проанализируйте как температура комментария (вероятность того, что он негативный) зависит от длины трэда (число комментариев под новостью). Определите какие тематики новостей самые негативные, а какие самые позитивные.  
4. __Необязательно:__  Можно обучить сентимент-классификатор на корпусе новостей из твиттера, но в качестве эмбедингов использовать ELMO слой, позаимствованный через tensor hub у лаборатории IPavlov. Можно не включать его в архитектуру, а просто прогнать через него выборку и сохранить. На вашем месте я бы усреднял каждый твитт по всем векторам и сохранял бы для каждого твитта только один вектор. 

Одним словом говоря, нас ждёт огромная работа. К счастью у вас есть тетрадки с семинаров, которые помогут вам в ней. 

__Внимание!__ сейчас мы будем насиловать опертивную память своего компьютера. Данные и модели будут гиговые, память будет быстро забиваться. Иногда тетрадка будет вылетать, а ядро умирать. Попытайтесь подгружать в неё только те данные, которые вам необходимы на данном этапе работы. Также рекомендуется сохранять промежуточные результаты, когда я рекомендую это сделать, чтобы впоcледствии не терять время попусту из-за умершего ядра. 

# 1. Знакомство с данными 

Давайте подгрузим все данные, которые у нас есть и подумаем как лучше с ними работать так, чтобы не запутаться и не сойти с ума. Все данные весят около 2 гигов. На самом деле я рекомендовал бы работать только с актуальными реально необходимыми для текущего куска задачи данными. 

In [1]:
import pickle

# Подгружаем все данные, что у нас есть 
with open('data/ria_news_2018.pickle', 'rb') as f:
    ria_news = pickle.load(f)
    
with open('data/ria_vk_posts', 'rb') as f:
    vk_news = pickle.load(f)
    
with open('data/ria_vk_comments', 'rb') as f:
    vk_comments = pickle.load(f)
    
print('Всего новостей за 2018 год:', len(ria_news))
print('Всего новостей в группе вк', len(vk_news))
print('Всего комментариев в группе вк', len(vk_comments))

Всего новостей за 2018 год: 201797
Всего новостей в группе вк 32277
Всего комментариев в группе вк 4040551


Каждая новость, пост, комментарий - это небольшой питоновский словарик. Посмотрим на примеры.

In [2]:
# Иньересующие нас поля: 
# category - рубрика с сайта
# date - дата публикации 
# keywords - ключевые слова
# shows - число просмотров новости 
# text - текст новости 
# title - заголовок новости
# href - ссылка на новость

ria_news[0]

{'category': 'В мире',
 'date': '2018-12-31',
 'day': '31',
 'href': '/20181231/1548961410.html',
 'image': 'https://cdn25.img.ria.ru/images/153303/06/1533030626_0:247:3501:2216_1036x0_80_0_0_291c92f1d31f7b2a613588e6a96164cb.jpg',
 'keywords': 'биньямин нетаньяху, израиль, в мире',
 'month': '12',
 'shows': '728',
 'snippet': 'Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24... РИА Новости, 31.12.2018',
 'tags': 'Биньямин Нетаньяху, Израиль, В мире',
 'text': 'МОСКВА, 31 дек - РИА Новости. Премьер-министр Израиля Биньямин Нетаньяху не намерен уходить в отставку в случае предъявления ему обвинений в коррупции, сообщает израильский телеканал i24 News. В начале декабря полиция рекомендовала генпрокуратуре предъявить обвинения в коррупции Нетаньяху и его супруге Саре по итогам расследования дела об их отношениях с руководством крупнейшей в стране телекоммуникационной компании "Безек"

In [3]:
# Интересующие нас поля: 
# date - дата поста (число секунд прошедшее с 1 января 1970 года)
# id - идентификатор поста 
# text - содержание поста
# title - заголовок новости 
# url - ссылка на новость
# likes - число лайков 
# comments - число комментов под постом

vk_news[10]

{'likes': 144,
 'reposts': 8,
 'comments': 151,
 'text': '',
 'date': 1549047725,
 'from_id': -15755094,
 'id': 24005074,
 'owner_id': -15755094,
 'url': 'https://ria.ru/20190201/1550257362.html',
 'title': 'Лукашенко поручил убрать из школьных программ "все лишнее"'}

In [4]:
# Интересующие нас поля: 
# date - дата поста (число секунд прошедшее с 1 января 1970 года)
# id - id комментария 
# post_id - id поста, к которому был оставлен комментарий 
# likes - число лайков на комментарии 
# text - текст комментария 

vk_comments[0]

{'id': 24003685,
 'from_id': 529161030,
 'date': 1549038419,
 'text': 'встану утром рано, выпью чашку ртути и пойду подохну в этом институте',
 'likes': 39,
 'post_id': 24003681,
 'group_id': '15755094'}

Познакомились? Теперь попробуйте побороть страх пустоты. Код придётся писать чуть ли не с нуля. 

# 2. Новые новости 

Тут всё просто. Нейросетку надо обучить по аналогии с тем, что мы делали с Евгением Онегиным на паре. Прошу вас, не ставить много экспериментов с выведением текстов после каждой эпохи. Один раз обучите нейросеть и выдайте фэйковую новсть только после последней эпохи. Выбор конкретной модификации Softmax поясните. 

In [ ]:
# Попытка разобраться в материалах с лекции поможет вам. Правда поможет. Это несложно.

# 3. Новостные категории

Теперь сложнее. Можно подглядывать в семинар, который был две недели назад. Там мы классифицировали отзывы по двум классам, тут надо будет классифицировать тексты по большому их числу. Попробуйте сначала решить какие категории вы точно не собираетесь предсказывать (встречаются в данных очень редко) и избавьтесь от них. Попробуйте предсказывать для каждой новости три самых вероятных категории. Редко бывает так, что для новости верна только одна категория. Это придаст дальнейшему анализу немного плюрализма. 

In [ ]:
# ваш код 

# Опциональная часть с температурой комментариев

Тут нужно будет понять какие новости публиковались в социальной сетке, а какие нет. Туда идёт не полный поток новостей, а только самое важное. После можно смотреть на категории и делать сентимент-анализ комментариев. 

In [ ]:
# ваш код 